In [2]:
import random
from my_lib import *
from IPython.display import clear_output

In [4]:
# Khởi tạo biến toàn cục
dang_nhap_status = False
user = ""

In [5]:
def quay_so():
    danh_sach_giai = {}

    ten_giai = ["GIẢI NHẤT", "GIẢI NHÌ", "GIẢI BA", "GIẢI TƯ", "GIẢI NĂM", "GIẢI SÁU", "GIẢI BẢY"]

    for giai in ten_giai:
        so = ""
        for i in range(5):
            so += str(random.randint(0, 9))  # Tạo một chữ số ngẫu nhiên từ 0 đến 9 và thêm vào số
        danh_sach_giai[giai] = so

    return danh_sach_giai



In [6]:
def in_danh_sach_giai(danh_sach_giai):
    print('-' * 20)
    print("---DANH SÁCH GIẢI---")
    for giai, so in danh_sach_giai.items():
        print(f"{giai}: {so}")
    print('-' * 20)

In [7]:
def nhap_so_lo():
    while True:
        so_lo = input("Nhập số lô mà bạn muốn cược (10-99): ")
        if so_lo.isdigit() and 10 <= int(so_lo) <= 99:
            return int(so_lo)
        else:
            print("Nhập sai định dạng, vui lòng nhập lại!")

In [8]:
def nhap_so_lo():
    while True:
        so_lo_input = input("Nhập các số lô bạn muốn chơi, cách nhau bằng dấu phẩy: ")
        danh_sach_so_lo = so_lo_input.split(",")

        so_lo_hop_le = True

        for so_lo in danh_sach_so_lo:
            if len(so_lo) != 2 or not so_lo.isdigit():
                print("Số lô nhập không hợp lệ. Vui lòng nhập lại.")
                so_lo_hop_le = False
                break

        if so_lo_hop_le:
            return danh_sach_so_lo


In [9]:
def nhap_tien_cuoc():
    while True:
        tien_cuoc = input("Nhập tiền cược mà bạn muốn cược (tiền cược < tổng tiền bạn có): ")
        if tien_cuoc.isdigit() and int(tien_cuoc) < tong_tien:
            return int(tien_cuoc)
        else:
            print("Nhập sai định dạng hoặc quá tổng tiền. Nhập lại")

In [27]:
cap_nhat_tien_choi_lo("dang",123123123)

In [63]:
from datetime import datetime

def cap_nhat_tien_choi_lo(username, tong_tien):
    try:
        du_lieu_tai_khoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
        if du_lieu_tai_khoan:
            for i, tai_khoan in enumerate(du_lieu_tai_khoan):
                if tai_khoan[0] == username:
                    du_lieu_tai_khoan[i][2] = str(tong_tien)  # Cập nhật số tiền
                    ghi_file([','.join(tai_khoan) + '\n' for tai_khoan in du_lieu_tai_khoan], PATH_DATA_TAI_KHOAN, "w")
                    return
    except Exception as e:
        print(f"Có lỗi xảy ra khi cập nhật thông tin tài khoản: {e}")


def luu_thong_tin_choi_lo(username, danh_sach_so_lo, tien_cuoc, danh_sach_giai, so_lo_trung, tien_thang, tien_thua):
    try:
        thoi_gian_choi = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        thong_tin = f"{thoi_gian_choi},{username},{','.join(danh_sach_so_lo)},{tien_cuoc}," \
                    f"{','.join(danh_sach_giai.values())},{','.join(so_lo_trung)},{tien_thang},{tien_thua}\n"
        
        ghi_file(thong_tin, PATH_DATA_CHOI_LO, "a")
        print("Thông tin về lần chơi lô đã được lưu.")
        
        # Cập nhật số tiền thắng/thua vào tài khoản

    except Exception as e:
        print(f"Có lỗi xảy ra khi lưu thông tin: {e}")

def game_quay_so(username, danh_sach_so_lo, tien_cuoc):
    tong_tien = int(lay_thong_tin_tai_khoan(username)[2])

    danh_sach_giai = quay_so()
    
    in_danh_sach_giai(danh_sach_giai)
    dem_so_lo = 0
    so_lo_trung = []  # Danh sách lưu trữ số lô trúng

    print("Các số lô bạn chơi là:", danh_sach_so_lo)
    for giai, so_giai in danh_sach_giai.items():
        for so_lo in danh_sach_so_lo:
            if so_lo == so_giai[-2:]:
                dem_so_lo += 1
                so_lo_trung.append(so_lo)

    tien_thang = 0  # Khởi tạo tien_thang
    tien_thua = 0   # Khởi tạo tien_thua

    if dem_so_lo > 0:
        tien_thang = tien_cuoc * dem_so_lo * 70
        tien_thua = tien_cuoc * (len(danh_sach_so_lo) - len(so_lo_trung))

        tong_tien += tien_thang - tien_thua

        print(f"Bạn đã trúng {dem_so_lo} nháy!")
        print("Các số lô trúng:", ", ".join(so_lo_trung))
        print(f"Số tiền bạn trúng là {tien_thang}")
        print(f"Số tiền bạn thua là {tien_thua}")
        print(f"Tổng tiền bạn có là {tong_tien}")
    else:
        tien_thua = tien_cuoc * len(danh_sach_so_lo)
        tong_tien -= tien_thua

        print("Bạn đã thua lô!")
        print(f"Số tiền bạn thua là {tien_thua}")
        print(f"Tổng tiền bạn còn là {tong_tien}")

    # Lưu thông tin về lần chơi lô vào file
    luu_thong_tin_choi_lo(username, danh_sach_so_lo, tien_cuoc, danh_sach_giai, so_lo_trung, tien_thang, tien_thua)
    cap_nhat_tien_choi_lo(username, tong_tien)

# Sử dụng hàm để chơi lô
username = "dang"  # Thay thế bằng username thực tế của user
danh_sach_so_lo = ["12", "21", "23", "22", "44"]  # Danh sách số lô cược
tien_cuoc = 10000  # Số tiền cược
game_quay_so(username, danh_sach_so_lo, tien_cuoc)


--------------------
---DANH SÁCH GIẢI---
GIẢI NHẤT: 72991
GIẢI NHÌ: 05104
GIẢI BA: 63390
GIẢI TƯ: 40997
GIẢI NĂM: 00827
GIẢI SÁU: 52646
GIẢI BẢY: 39707
--------------------
Các số lô bạn chơi là: ['12', '21', '23', '22', '44']
Bạn đã thua lô!
Số tiền bạn thua là 50000
Tổng tiền bạn còn là 10910000
Thông tin về lần chơi lô đã được lưu.


In [24]:
def nap_tien(username, so_tien_nap):
    # Đọc dữ liệu từ file tài khoản
    lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
    if lst_taikhoan is None:
        print("Không thể nạp tiền. Có lỗi xảy ra khi đọc file tài khoản.")
        return

    # Tìm tài khoản trong danh sách
    for tai_khoan_info in lst_taikhoan:
        if tai_khoan_info[0] == username:
            # Cộng số tiền nạp vào số tiền của tài khoản
            tai_khoan_info[2] = str(int(tai_khoan_info[2]) + so_tien_nap)
            # Ghi lại dữ liệu mới vào file
            ghi_file([','.join(tai_khoan) + '\n' for tai_khoan in lst_taikhoan], PATH_DATA_TAI_KHOAN, "w")
            print(f"Đã nạp {so_tien_nap} vào tài khoản {username}.")
            return

    print("Không tìm thấy tài khoản.")
    return

# Sử dụng hàm để nạp tiền
username = "dang"  # Thay thế bằng username thực tế của user
so_tien_nap = 50000  # Số tiền cần nạp
nap_tien(username, so_tien_nap)


Đã nạp 50000 vào tài khoản dang.


In [13]:
def lay_thong_tin_tai_khoan(username):
    try:
        du_lieu_tai_khoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
        if du_lieu_tai_khoan:
            for tai_khoan in du_lieu_tai_khoan:
                if tai_khoan[0] == username:
                    return tai_khoan
        else:
            return None
    except Exception as e:
        print(f"Có lỗi xảy ra: {e}")
        return None

In [11]:
def kiem_tra_admin(username):
    if username == "admin":
        return 1
    else:
        return 0 


In [12]:
def kiem_tra_tai_khoan(username, password):
    global dang_nhap_status, user
    # Đọc dữ liệu từ file tài khoản
    lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
    if lst_taikhoan is None:
        print("Không thể đăng nhập. Có lỗi xảy ra khi đọc file tài khoản.")
        return

    for tai_khoan in lst_taikhoan:
        if tai_khoan[0] == username and tai_khoan[1] == password:
            dang_nhap_status = True
            user = username
            return kiem_tra_admin(username)


In [13]:
def kiem_tra_ton_tai_tai_khoan(username):
    # Đọc dữ liệu từ file tài khoản
    lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
    if lst_taikhoan is None:
        return False

    # Kiểm tra xem tài khoản đã tồn tại chưa
    for tai_khoan in lst_taikhoan:
        if tai_khoan[0] == username:
            return True
    
    return False


In [14]:
import my_lib
lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
lst_taikhoan

[['admin', 'admin', '1000000'], ['dang', '654321', '160000']]

In [15]:
def doi_mat_khau(username, password_cu, password_moi):
    # Đọc dữ liệu từ file tài khoản
    lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
    if lst_taikhoan is None:
        print("Không thể đổi mật khẩu. Có lỗi xảy ra khi đọc file tài khoản.")
        return

    # Tìm tài khoản trong danh sách và kiểm tra mật khẩu cũ
    for tai_khoan_info in lst_taikhoan:
        if tai_khoan_info[0] == username and tai_khoan_info[1] == password_cu:
            # Cập nhật mật khẩu mới
            tai_khoan_info[1] = password_moi
            # Ghi lại dữ liệu mới vào file
            ghi_file([','.join(tai_khoan) + '\n' for tai_khoan in lst_taikhoan], PATH_DATA_TAI_KHOAN, "w")
            print("Đổi mật khẩu thành công.")
            return

    print("Không tìm thấy tài khoản hoặc mật khẩu cũ không đúng.")
    return

# Sử dụng hàm để đổi mật khẩu
username = "dang"  # Thay thế bằng username thực tế của user
password_cu = "123456"  # Mật khẩu cũ
password_moi = "654321"  # Mật khẩu mới
doi_mat_khau(username, password_cu, password_moi)


Không tìm thấy tài khoản hoặc mật khẩu cũ không đúng.


In [16]:
def dang_nhap():
    global dang_nhap_status
    print("---GAME LÔ ĐỀ HỌC---")
    print("Vui lòng đăng nhập để chơi game")
    
    while True:
        username = input("Nhập username: ")
        password = input("Nhập password: ")

        if kiem_tra_tai_khoan(username, password) in (0,1):
            dang_nhap_status = True
            break
        else:
            print("Sai tên người dùng hoặc mật khẩu, vui lòng thử lại.")
    clear_output()
    menu()

In [17]:
def menu():
    print("GAME LÔ ĐỀ HỌC")
    print("1. Chơi lô")
    print("2. Đổi mật khẩu")
    print("3. Thống kê")
    print("4. Đăng xuất")

In [18]:
def menu_admin():
    print("ADMIN GAME LÔ ĐỀ HỌC")
    print("1. Tạo tài khoản")
    print("2. Nạp tiền tài khoản")
    print("3. Thống kê")
    print("4. Đăng xuất")
    

In [19]:
def main():
    global dang_nhap_status
    if dang_nhap_status == False:
        dang_nhap()
    else:
        if user == "admin":
            menu_admin()
        else:
            menu()
    if user == "admin":
        while True:
            lua_chon = int(input("Bạn chọn: "))
            match lua_chon:
                case 1:
                    print(1)
                    break
                case 2:
                    print(2)
                    break
                case 3:
                    print(3)
                    break
                case 4:
                    dang_nhap_status = False
                    print("Chào bạn, hẹn gặp lại!")
                    break
                case _:
                    print("Nhập sai lựa chọn, vui lòng nhập lại !")
    else:
        while True:
            lua_chon = int(input("Bạn chọn: "))
            match lua_chon:
                case 1:
                    clear_output(wait=True)
                    while True:
                        tien_cuoc = nhap_tien_cuoc()
                        so_lo = nhap_so_lo()
                        game_quay_so(so_lo, tien_cuoc)
                        choi_tiep = int(input("Bạn có muốn chơi tiếp không? (0: không, 1: có)"))
                        if choi_tiep == 0:
                            break
                    clear_output()
                    main()
                    break
                case 2:
                    clear_output()
                    nap_tien()
                    clear_output(wait=True)
                    main()
                    break
                case 3:
                    print(3)
                    break    
                case 4:
                    dang_nhap_status = False
                    print("Chào bạn, hẹn gặp lại!")
                    break
                case _:
                    print("Nhập sai lựa chọn, vui lòng nhập lại !")

In [54]:
main()

GAME LÔ ĐỀ HỌC
1. Chơi lô
2. Đổi mật khẩu
3. Thống kê
4. Đăng xuất


Bạn chọn:  4


Chào bạn, hẹn gặp lại!


In [20]:
def tao_tai_khoan(username, password, tong_tien):
    if check_tai_khoan(username):
        user_info = f"{username},{password},{tong_tien}\n"
        ghi_file([user_info], PATH_DATA_TAI_KHOAN, "a")
        print(f"Tạo tài khoản {username} thành công")
    else:
        print("Trùng username")


In [21]:
def xoa_tai_khoan(username):
    if kiem_tra_admin(username):
        print("Không được xóa tài khoản admin")
    else:
        lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
        for tai_khoan_info in lst_taikhoan:
            if tai_khoan_info[0] == username:
                lst_taikhoan.remove(tai_khoan_info)
                ghi_file([','.join(tai_khoan) + '\n' for tai_khoan in lst_taikhoan], PATH_DATA_TAI_KHOAN, "w")
                return "Xóa tài khoản thành công"

        return "User không tồn tại"

# Sử dụng hàm để xóa tài khoản
username_can_xoa = "user1234"
print(xoa_tai_khoan(username_can_xoa))


User không tồn tại


In [22]:
lst_taikhoan = doc_file(PATH_DATA_TAI_KHOAN, "r")

lst_taikhoan

[['admin', 'admin', '1000000'], ['dang', '654321', '160000']]

In [4]:
def thong_ke_user(username):
    try:
        du_lieu_choi_lo = doc_file(PATH_DATA_CHOI_LO, "r")
        so_lan_choi = 0
        tong_tien_thang = 0
        tong_tien_thua = 0
        so_lan_thang = 0
        so_lan_thua = 0

        for choi_lo in du_lieu_choi_lo:
            if choi_lo[1] == username:
                so_lan_choi += 1
                tong_tien_thang += int(choi_lo[11])
                tong_tien_thua += int(choi_lo[12])
                if int(choi_lo[-2]) >  0:
                    so_lan_thang += 1
                else:
                    so_lan_thua += 1
        ti_le_thang = round(so_lan_thang / so_lan_thua, 2) if so_lan_thua > 0 else 0

        # In kết quả
        print("Thông kê cho người dùng", username)
        print("Số lần chơi lô:", so_lan_choi)
        print("Tổng tiền chơi lô thắng:", tong_tien_thang)
        print("Tổng tiền chơi lô thua:", tong_tien_thua)
        print("Tỉ lệ chơi lô thắng/thua:", ti_le_thang)

    except Exception as e:
        print(f"Có lỗi xảy ra khi thực hiện thống kê: {e}")
thong_ke_user("dang")

Thông kê cho người dùng dang
Số lần chơi lô: 4
Tổng tiền chơi lô thắng: 280746
Tổng tiền chơi lô thua: 149684
Tỉ lệ chơi lô thắng/thua: 0.33


In [ ]:
def thong_ke_admin():
    try:
        # Đọc dữ liệu từ file choilo.txt
        du_lieu_choi_lo = doc_file(PATH_DATA_CHOI_LO, "r")
        so_luong_tai_khoan = len(doc_file(PATH_DATA_TAI_KHOAN, "r"))
        tong_so_lan_choi_lo = len(du_lieu_choi_lo)
        tong_tien_thang = sum(int(choi_lo[11]) for choi_lo in du_lieu_choi_lo)
        tong_tien_thua = sum(int(choi_lo[12]) for choi_lo in du_lieu_choi_lo)
        so_lan_thang = sum(1 for choi_lo in du_lieu_choi_lo if int(choi_lo[11]) > 0)
        so_lan_thua = tong_so_lan_choi_lo - so_lan_thang

        # Tính tỷ lệ thắng/thua
        ti_le_thang_thua = round(so_lan_thang / so_lan_thua, 2) if so_lan_thua > 0 else 0

        # In kết quả thống kê
        print("Thống kê tổng hợp:")
        print("Số lượng tài khoản:", so_luong_tai_khoan)
        print("Tổng số lượt chơi lô:", tong_so_lan_choi_lo)
        print("Tổng tiền chơi lô thắng:", tong_tien_thang)
        print("Tổng tiền chơi lô thua:", tong_tien_thua)
        print("Tỉ lệ thắng/thua:", ti_le_thang_thua)

    except Exception as e:
        print(f"Có lỗi xảy ra khi thực hiện thống kê tổng hợp: {e}")
